# Step 1: Brief description of the problem and data
In this Kaggle competition project, the task is to build a GAN (generative adversarial network) that generates 7,000 to 10,000 Monet-style images.

From Kaggle:
The dataset contains four directories: `monet_tfrec`, `photo_tfrec`, `monet_jpg`, and `photo_jpg`. The `monet_tfrec` and `monet_jpg` directories contain the same painting images, and the `photo_tfrec` and `photo_jpg` directories contain the same photos.
* The `monet` directories contain Monet paintings. Use these images to train your model.
* The `photo` directories contain photos. Add Monet-style to these images and submit your generated jpeg images as a zip file. Other photos outside of this dataset can be transformed but keep your submission file limited to 10,000 images.

Note: Monet-style art can be created from scratch using other GAN architectures like DCGAN. The submitted image files do not necessarily have to be transformed photos.

Kaggle describes the files provided as follows:
* **monet_jpg** - 300 Monet paintings sized 256x256 in JPEG format
* **monet_tfrec** - 300 Monet paintings sized 256x256 in TFRecord format
* **photo_jpg** - 7028 photos sized 256x256 in JPEG format
* **photo_tfrec** - 7028 photos sized 256x256 in TFRecord format

The output of this project must be called `images.zip` and contain 7,000 to 10,000 images sized 256x256.

# Step 2: Exploratory Data Analysis (EDA) - Inspect, Visualize and Clean the Data
Show a few visualizations like histograms. Describe any data cleaning procedures. Based on your EDA, what is your plan of analysis?

# Step 3: Model Architecture
Describe your model architecture and reasoning for why you believe that specific architecture would be suitable for this problem.

Since we did not learn NLP-specific techniques such as word embeddings in the lectures, we recommend looking at Kaggle tutorials, discussion boards, and code examples posted for this challenge.  You can use any resources needed, but make sure you “demonstrate” you understood by including explanations in your own words. Also importantly, please have a reference list at the end of the report.

There are many methods to process texts to matrix form (word embedding), including TF-IDF, GloVe, Word2Vec, etc. Pick a strategy and process the raw texts to word embedding. Briefly explain the method(s) and how they work in your own words.

Build and train your sequential neural network model (You may use any RNN family neural network, including advanced architectures LSTM, GRU, bidirectional RNN, etc.).

# Step 4: Results and Analysis
Run hyperparameter tuning, try different architectures for comparison, apply techniques to improve training or performance, and discuss what helped.

Includes results with tables and figures. There is an analysis of why or why not something worked well, troubleshooting, and a hyperparameter optimization procedure summary.

# Step 5: Conclusion
Discuss and interpret results as well as learnings and takeaways. What did and did not help improve the performance of your models? What improvements could you try in the future?